In [1]:
import xarray as xr
from extremeweatherbench import inputs
from skimage.measure import find_contours
from skimage.feature import peak_local_max
from typing import Sequence
import numpy as np

In [2]:
hres_forecast = inputs.ZarrForecast(
    source="gs://weatherbench2/datasets/hres/2016-2022-0012-1440x721.zarr",
    variables=["surface_air_temperature", "surface_eastward_wind", "surface_northward_wind", "eastward_wind", "northward_wind", "lead_time", "init_time", "air_pressure_at_mean_sea_level", "surface_wind_speed"],
    variable_mapping={
        "2m_temperature": "surface_air_temperature",
        "10m_u_component_of_wind": "surface_eastward_wind",
        "10m_v_component_of_wind": "surface_northward_wind",
        "u_component_of_wind": "eastward_wind",
        "v_component_of_wind": "northward_wind",
        "prediction_timedelta": "lead_time",
        "time": "init_time",
        "lead_time": "prediction_timedelta",
        "mean_sea_level_pressure": "air_pressure_at_mean_sea_level",
        "10m_wind_speed": "surface_wind_speed",
    },
    storage_options={"remote_options": {"anon": True}},
)

In [3]:
forecast_dask_ds = hres_forecast.open_and_maybe_preprocess_data_from_source()

In [4]:
forecast_dask_ds

<xarray.Dataset> Size: 87TB
Dimensions:                   (time: 5134, prediction_timedelta: 41,
                               latitude: 721, longitude: 1440, level: 13)
Coordinates:
  * latitude                  (latitude) float32 3kB -90.0 -89.75 ... 89.75 90.0
  * level                     (level) int32 52B 50 100 150 200 ... 850 925 1000
  * longitude                 (longitude) float32 6kB 0.0 0.25 ... 359.5 359.8
  * prediction_timedelta      (prediction_timedelta) timedelta64[ns] 328B 00:...
  * time                      (time) datetime64[ns] 41kB 2016-01-01 ... 2023-...
Data variables: (12/16)
    10m_u_component_of_wind   (time, prediction_timedelta, latitude, longitude) float32 874GB dask.array<chunksize=(1, 1, 721, 1440), meta=np.ndarray>
    10m_v_component_of_wind   (time, prediction_timedelta, latitude, longitude) float32 874GB dask.array<chunksize=(1, 1, 721, 1440), meta=np.ndarray>
    10m_wind_speed            (time, prediction_timedelta, latitude, longitude) float32 874GB dask.array<chunksize=(1, 1, 721, 1440), meta=np.ndarray>
    2m_temperature            (time, prediction_timedelta, latitude, longitude) float32 874GB dask.array<chunksize=(1, 1, 721, 1440), meta=np.ndarray>
    geopotential              (time, prediction_timedelta, level, latitude, longitude) float32 11TB dask.array<chunksize=(1, 1, 13, 721, 1440), meta=np.ndarray>
    mean_sea_level_pressure   (time, prediction_timedelta, latitude, longitude) float32 874GB dask.array<chunksize=(1, 1, 721, 1440), meta=np.ndarray>
    ...                        ...
    total_precipitation_24hr  (time, prediction_timedelta, latitude, longitude) float32 874GB dask.array<chunksize=(1, 1, 721, 1440), meta=np.ndarray>
    total_precipitation_6hr   (time, prediction_timedelta, latitude, longitude) float32 874GB dask.array<chunksize=(1, 1, 721, 1440), meta=np.ndarray>
    u_component_of_wind       (time, prediction_timedelta, level, latitude, longitude) float32 11TB dask.array<chunksize=(1, 1, 13, 721, 1440), meta=np.ndarray>
    v_component_of_wind       (time, prediction_timedelta, level, latitude, longitude) float32 11TB dask.array<chunksize=(1, 1, 13, 721, 1440), meta=np.ndarray>
    vertical_velocity         (time, prediction_timedelta, level, latitude, longitude) float32 11TB dask.array<chunksize=(1, 1, 13, 721, 1440), meta=np.ndarray>
    wind_speed                (time, prediction_timedelta, level, latitude, longitude) float32 11TB dask.array<chunksize=(1, 1, 13, 721, 1440), meta=np.ndarray>

In [ ]:
# - case_id_number: 225
#   title: Ida
#   start_date: 2021-08-24 00:00:00
#   end_date: 2021-09-06 00:00:00
#   location:
#     type: bounded_region
#     parameters:
#       latitude_min: 14.3
#       latitude_max: 51.0
#       longitude_min: 266.8
#       longitude_max: 299.9

In [17]:
slp_time = (forecast_dask_ds['mean_sea_level_pressure']
            .sel(time=slice("2021-08-14 00:00:00","2021-09-06 00:00:00"))
            .sel(latitude=slice(14.3, 51.0), longitude=slice(266.8, 299.9))
            )
slp_time

<xarray.DataArray 'mean_sea_level_pressure' (time: 47,
                                             prediction_timedelta: 41,
                                             latitude: 147, longitude: 132)> Size: 150MB
dask.array<getitem, shape=(47, 41, 147, 132), dtype=float32, chunksize=(1, 1, 147, 132), chunktype=numpy.ndarray>
Coordinates:
  * latitude              (latitude) float32 588B 14.5 14.75 15.0 ... 50.75 51.0
  * longitude             (longitude) float32 528B 267.0 267.2 ... 299.5 299.8
  * prediction_timedelta  (prediction_timedelta) timedelta64[ns] 328B 00:00:0...
  * time                  (time) datetime64[ns] 376B 2021-08-14 ... 2021-09-06
Attributes:
    long_name:      Mean sea level pressure
    short_name:     msl
    standard_name:  air_pressure_at_mean_sea_level
    units:          Pa

In [18]:
def peak_mask_wrapper(array, min_distance: int = 12):
    """Wrapper that returns a boolean mask of peaks instead of coordinates."""
    print(f"Processing array shape {array.shape}")
    
    # Get peak coordinates
    peaks = peak_local_max(
        -array,  # negative for minima (like SLP)
        min_distance=min_distance,
        exclude_border=0
    )
    
    # Create boolean mask
    mask = np.zeros(array.shape, dtype=bool)
    if len(peaks) > 0:
        mask[tuple(peaks.T)] = True
    return mask

# Apply with xr.apply_ufunc
peak_mask_dask = xr.apply_ufunc(
    peak_mask_wrapper,
    slp_time,
    input_core_dims=[['latitude', 'longitude']],
    output_core_dims=[['latitude', 'longitude']],  # same shape as input
    dask='parallelized',
    output_dtypes=[bool],
    kwargs={'min_distance': 12},  # pass additional arguments
)
peak_mask_dask
result = peak_mask_dask.compute()
result

<xarray.DataArray 'mean_sea_level_pressure' (time: 47,
                                             prediction_timedelta: 41,
                                             latitude: 147, longitude: 132)> Size: 37MB
array([[[[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False,  True, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]],

        [[False, False, False, ..., False, False, False],
         [ True, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]],

        [[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
...
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]],

        [[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]],

        [[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]]]],
      shape=(47, 41, 147, 132))
Coordinates:
  * latitude              (latitude) float32 588B 14.5 14.75 15.0 ... 50.75 51.0
  * longitude             (longitude) float32 528B 267.0 267.2 ... 299.5 299.8
  * prediction_timedelta  (prediction_timedelta) timedelta64[ns] 328B 00:00:0...
  * time                  (time) datetime64[ns] 376B 2021-08-14 ... 2021-09-06

In [ ]:

def find_contours_from_point_specified_field(field: xr.DataArray, point: tuple[float, float], level: float) -> Sequence[tuple[float, float]]:
    field_at_point = field - field.isel(latitude=point[0], longitude=point[1])
    contours = find_contours(field_at_point.values, level=level, positive_orientation='high')
    return contours

candidate_slp_points = peak_local_max(
    -(slp_time.values),
    min_distance=min_distance,
    # exclude_border=0 required as points <= min_distance from border are not included
    exclude_border=0
)